In [11]:
import os
from dotenv import load_dotenv
from docx import Document
from sentence_transformers import SentenceTransformer
from pymilvus import Collection, connections
from openai import OpenAI

load_dotenv(override=True, dotenv_path="../.env.local")

openai_key = os.getenv("OPENAI_API_KEY")
milvus_uri = os.getenv("MILVUS_URI")
milvus_api_key = os.getenv("MILVUS_API_KEY")

print(openai_key[:3])
print(milvus_uri[:3])
print(milvus_api_key[:3])


sk-
htt
b98


In [ ]:

class PolicyRAG:
    def __init__(self, milvus_uri, milvus_token, openai_key):
        # Initialize all your "tools" in one place
        print("Initializing RAG System...")
        self.model = SentenceTransformer("all-MiniLM-L6-v2")
        self.ai_client = OpenAI(api_key=openai_key)
        
        # Connect to Milvus
        connections.connect("default", uri=milvus_uri, token=milvus_token)
        self.collection = Collection("policy_docs_collection")
        self.collection.load()
        print("System Ready.")

    def get_context(self, question, limit=3):
        """Step 1: Search Milvus for relevant snippets"""
        embedding = self.model.encode(question).tolist()
        
        results = self.collection.search(
            data=[embedding],
            anns_field="embedding",
            param={"metric_type": "COSINE", "params": {"nprobe": 10}},
            limit=limit,
            output_fields=["title", "content"]
        )
        
        context_text = ""
        sources = []
        for hit in results[0]:
            context_text += f"\nSource: {hit.entity.get('title')}\nContent: {hit.entity.get('content')}\n"
            sources.append(hit.entity.get('title'))
        
        return context_text, sources

    def ask(self, question):
        """Step 2: Use the context to get an answer from OpenAI"""
        context, sources = self.get_context(question)
        
        response = self.ai_client.chat.completions.create(
            model="gpt-4o",
            messages=[
                {"role": "system", "content": "Answer only using the provided policy context."},
                {"role": "user", "content": f"Context: {context}\n\nQuestion: {question}"}
            ],
            temperature=0
        )
        
        return response.choices[0].message.content, list(set(sources))




In [14]:
# --- HOW TO USE IT ---
# Create the "Engine" once
rag_engine = PolicyRAG(
    milvus_uri,
    milvus_api_key,
    openai_key)

question="How do I maintain my SpeakEQ account?"
# Now you can just ask questions simply!
answer, files = rag_engine.ask(question)

print(f"\nAI ANSWER:\n{answer}")
print(f"FILES USED: {files}")

Initializing RAG System...
System Ready.

AI ANSWER:
To maintain your SpeakEQ account, you can manage your subscription settings, including cancellation, through your account settings or your device's app store. If you have any questions or need assistance, you can contact support at support@speakeq.
FILES USED: ['Terms_ And_Conditions.docx']
